## Test Cases for DDL used by Apache Spark
**Spark SQL 3.3 DDL and DML Reference** 
https://spark.apache.org/docs/3.3.0/sql-ref-syntax.html#ddl-statements

**<mark>Excludes</mark>:**
1. Use Database
2. Functions (Create, Drop Function)

To store these results configure data <mark>**storage account and container**</mark>.

In [ ]:
!pip install unittest-xml-reporting xmltodict

## Configure Result Storage Location

In [ ]:
storage_account=""
result_container=""

## Initialize Common Variables for the test run

In [ ]:
import time

# Don't change these variables
TEST_SUITE= "SPARK_SQL_DDL"
RESULT_FILE_NAME="ddl_test_result.parquet"
RAW_RESULT_FILE_NAME="raw_ddl_test_result.parquet"
# Test Run ID
TEST_RUN_ID= round(time.time()*1000)
# Test platform
PLATFORM = "nameoftheplatform"
# Prefix for all tables
PREFIX = PLATFORM
SUFFIX = TEST_RUN_ID
# Spark SQL function
sql=spark.sql

### Set Common Spark Configurations

In [ ]:
sql("set hive.exec.dynamic.partition.mode=nonstrict")

### DDL - Database
1. Create database
2. Alter database - DB Properties
3. Describe database
4. Alter database location

In [ ]:
import unittest

TEST_DATABASE_NAME = f"sparksql_spec_db_{SUFFIX}"
DB_LOCATION = f"Files/sparksql_spec_ref_synapse/{SUFFIX}"

class DDLDatabaseTest(unittest.TestCase):
    """Database related test cases"""
    def test_ddl_database_00_create_database(self):
        """create database at location"""
        sql_cmd = f"CREATE DATABASE IF NOT EXISTS {PREFIX}_{TEST_DATABASE_NAME}_{SUFFIX} COMMENT 'This is spark sql specification database' LOCATION '{DB_LOCATION}' \
                        WITH DBPROPERTIES (ID=001, Name='User')"
        try:
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE DATABASE','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_database_01_alter_database_loction(self):
        """alter database at location"""
        sql_cmd = f"ALTER DATABASE {PREFIX}_{TEST_DATABASE_NAME}_{SUFFIX} SET LOCATION 'sparksql_spec_ref_synapse_newlocation'"
        try:
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'ALTER DATABASE LOCATION','status':'fail'}
            self.fail(f"{msg}")
        
    def test_ddl_database_02_alter_database_prop(self):
        """alter database properties"""
        sql_cmd = f"ALTER DATABASE {PREFIX}_{TEST_DATABASE_NAME}_{SUFFIX} SET DBPROPERTIES ('Edited-by' = 'NewUser', 'Edit-date' = '10/10/2023')"
        try:
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'ALTER DATABASE PROP','status':'fail'}
            self.fail(f"{msg}")

## DDL - Create Table 
1. Using data source
2. Using Hive Format
3. Using Like

In [ ]:
import unittest

class DDLCreateTableTest(unittest.TestCase):

    list_of_tables_to_remove=[]
    parquet_table_name=f"{PREFIX}_student_parquet_{SUFFIX}"
    csv_table_name= f"{PREFIX}_student_csv_{SUFFIX}"
    orc_table_name = f"{PREFIX}_Student_hive_orc_{SUFFIX}"

    """Create table test cases"""
    def test_ddl_create_table_001_create_table_datasource(self):
        table_name = f"{PREFIX}_student_delta_table_{SUFFIX}"
        """CREATE DATASOURCE DELTA TABLE"""
        sql_cmd = f"CREATE TABLE {table_name} (id INT, name STRING, age INT) \
                     USING DELTA PARTITIONED BY (age)"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE DATASOURCE DELTA TABLE','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_002_create_table_partition_bucketed_parquet(self):
        """Create partitioned and bucketed table parquet"""
        sql_cmd = f"CREATE TABLE {self.parquet_table_name} (id INT, name STRING, age INT) \
                      USING parquet PARTITIONED BY (age) CLUSTERED BY (Id) INTO 4 buckets"
        try:
            self.list_of_tables_to_remove.append(self.parquet_table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE USING parquet PARTITIONED BY CLUSTERED BY buckets','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_003_create_table_partition_bucketed_csv(self):
        """Create partitioned and bucketed table csv"""
        sql_cmd = f"CREATE TABLE {self.csv_table_name} (id INT, name STRING, age INT) \
                   USING csv PARTITIONED BY (age) CLUSTERED BY (Id) INTO 4 buckets"
        try:
            self.list_of_tables_to_remove.append(self.csv_table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE USING csv PARTITIONED BY CLUSTERED BY buckets','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_004_create_table_ctas_cte(self):
        """Create bucketed table through CTAS and CTE using parquet table"""
        table_name=f"{PREFIX}_student_bucket_ctas_cte_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} USING parquet CLUSTERED BY (id) INTO 4 buckets ( \
                WITH {PREFIX}_tmpTable_{SUFFIX} AS ( \
                    SELECT * FROM {self.parquet_table_name} WHERE id > 100 \
                ) \
                SELECT * FROM {PREFIX}_tmpTable_{SUFFIX} \
              );"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE USING parquet CLUSTERED BY through CTAS and CTE','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_005_create_table_like_from_existing_table(self):
        """Create table like using existing table"""
        table_name= f"{PREFIX}_Student_Dupli_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} like {self.parquet_table_name};"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE like using existing table','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_006_create_table_like_from_existing_table(self):
        """Create table like using existing table with datasource"""
        table_name=f"{PREFIX}_Student_Dupli_datasource_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} like {self.csv_table_name} using CSV;"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE like using existing table using datasource','status':'fail'}
            self.fail(f"{msg}")
    
    def test_ddl_create_table_007_create_table_like_row_format(self):
        """Create table like using existing table with rowformat"""
        table_name=f"{PREFIX}_student_Dupli_like_rowformat_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} like {self.csv_table_name} \
                  ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' \
                     STORED AS TEXTFILE TBLPROPERTIES ('createdby'='Test');"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE like using existing table with row format','status':'fail'}
            self.fail(f"{msg}")
    
    def test_ddl_create_table_008_create_table_hive_format_orc(self):
        """Create table Hive format ORC"""
        sql_cmd = f"CREATE TABLE {self.orc_table_name}(id INT, name STRING, age INT) \
                    COMMENT 'this is a comment' \
                    TBLPROPERTIES ('user'='testcases') \
                    STORED AS ORC;"
        try:
            self.list_of_tables_to_remove.append(self.orc_table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE Format ORC','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_009_create_table_hive_format_orc_from_another_table(self):
        """Create table Hive format ORC use data from another table"""
        table_name =f"{PREFIX}_Student_hive_orc_copy_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} STORED AS ORC \
                    AS SELECT * FROM {self.orc_table_name};"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE ORC from another table','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_010_create_table_hive_format_orc_partitioned_table(self):
        """Create table Hive format ORC partitioned table"""
        table_name = f"{PREFIX}_Student_hive_orc_partition_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} (id INT, name STRING) \
                    STORED AS ORC \
                    PARTITIONED BY (age INT);"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'Hive:CREATE TABLE ORC with partition','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_011_create_table_hive_format_text(self):
        """Create table Hive Row Format and file format"""
        table_name = f"{PREFIX}_Student_hive_text_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} (id INT, name STRING, age INT) \
                    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','\
                     STORED AS TEXTFILE;"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE TABLE Row Format and file format','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_create_table_012_create_table_hive_format_cluster_by(self):
        """Create table Hive clustered_by bucket table without SORTED BY, it is working with Apache Spark"""
        # for example CREATE TABLE test (id INT, name STRING, age INT) CLUSTERED BY (ID) INTO 4 BUCKETS STORED AS ORC"
        table_name = f"{PREFIX}_student_clustered_by_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} (id INT, name STRING, age INT) \
                    CLUSTERED BY (ID) \
                    INTO 4 BUCKETS \
                    STORED AS ORC;"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'Create table clustered_by bucket table without SORTED BY','status':'fail'}
            self.fail(f"{msg}")

    def test_create_table_013_create_table_hive_format_cluster_by_sorted_by(self):
        """Create table Hive clustered_by bucket table with SORTED BY"""
        table_name=f"{PREFIX}_student_clustered_by_sorted_by{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} (id INT, name STRING) \
                    PARTITIONED BY (YEAR STRING) \
                    CLUSTERED BY (ID, NAME) \
                    SORTED BY (ID ASC) \
                    INTO 3 BUCKETS \
                    STORED AS ORC;"
        try:
            self.list_of_tables_to_remove.append(table_name)
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'Create table clustered_by bucket table with SORTED BY','status':'fail'}
            self.fail(f"{msg}")

    @classmethod
    def tearDownClass(cls):
        """tear down"""
        for table in cls.list_of_tables_to_remove:
            try:
                sql(f"DROP TABLE IF EXISTS {table}")
            except Exception as e:
                print(f"table drop failed: {table}")

### DDL - Alter Table

In [ ]:
import unittest
from pyspark.sql.utils import AnalysisException

class DDLAlterTableTest(unittest.TestCase):
    """Alter table test cases"""
    
    table_name = f"{PREFIX}_student_alter_table_{SUFFIX}"
    rename_table = f"{PREFIX}_student_alter_table_{SUFFIX}_copy"
    list_of_tables_to_remove=[]

    @classmethod
    def setUpClass(cls):
        sql_cmd = f"CREATE TABLE {cls.table_name} (id INT, name STRING) \
                        PARTITIONED BY (age INT)"
        try:
            sql(sql_cmd)
            sql(f"INSERT INTO {cls.table_name} VALUES \
                 (1,'a',10),(2,'b',20),(3,'c',30);")
        except Exception as ex:
            msg={'command':'AlterTable:Setup failed','status':'fail'}
            cls.fail(f"{msg}")

    
    def test_ddl_altertable_001_rename_table_name(self):
        """Rename Table"""
        sql_cmd = f"ALTER TABLE {self.table_name} RENAME TO {self.rename_table}"
        try:
            sql(sql_cmd)
            # this table to be removed during teardown
            self.list_of_tables_to_remove.append(self.rename_table)
            record_count=sql(f"SELECT * FROM {self.rename_table}").count()
            self.assertEqual(record_count,3)
            num_of_partition=sql(f"SHOW PARTITIONS {self.rename_table}").where("partition='age=10'").count()
            self.assertEqual(num_of_partition,1)
        except Exception as ex:
            msg={'command':'ALTER TABLE RENAME TO','status':'fail'}
            self.list_of_tables_to_remove.append(self.table_name)
            self.fail(f"{msg}")
    
    def test_ddl_altertable_002_rename_partition(self):
        """Rename Table Partition after renaming Table Name, this scenario is working with open source version"""
        sql_cmd = f"ALTER TABLE {self.rename_table} PARTITION (age='10') RENAME TO PARTITION (age='15')"
        try:
            sql(sql_cmd)
            num_of_partition=sql(f"SHOW PARTITIONS {self.rename_table}").where("partition='age=10'").count()
            self.assertEqual(num_of_partition,0)
        except Exception as ex:
            msg={'command':'ALTER TABLE PARTITION RENAME TO PARTITION','status':'fail'}
            self.list_of_tables_to_remove.append(self.table_name)
            self.fail(f"{msg}")

    def test_ddl_altertable_003_add_new_partitions(self):
        """Add multiple new partition"""
        sql_cmd = f"ALTER TABLE {self.rename_table} ADD IF NOT EXISTS PARTITION (age=18)"
        try:
            sql(sql_cmd)
            num_of_partition=sql(f"SHOW PARTITIONS {self.rename_table}").where("partition='age=18'").count()
            self.assertEqual(num_of_partition,2)
        except Exception as ex:
            msg={'command':'ALTER TABLE ADD PARTITION','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_altertable_004_drop_new_partition(self):
        """Remove a partition"""
        sql_cmd = f"ALTER TABLE {self.rename_table} DROP IF EXISTS PARTITION (age=18)"
        try:
            sql(sql_cmd)
            num_of_partition=sql(f"SHOW PARTITIONS {self.rename_table}").where("partition='age=18'").count()
            self.assertEqual(num_of_partition,0)
        except Exception as ex:
            msg={'command':'ALTER TABLE DROP PARTITION','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_altertable_005_add_column(self):
        """Add column """
        sql_cmd = f"ALTER TABLE {self.rename_table} ADD COLUMNS (LastName string, country string)"
        try:
            sql(sql_cmd)
            col_exist=sql(f"DESC {self.rename_table}").where("col_name='LastName'").count()
            self.assertEqual(col_exist,1)
        except Exception as ex:
            msg={'command':'ALTER TABLE ADD COLUMNS','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_altertable_006_alter_column_comments(self):
        """Rename column comments"""
        sql_cmd = f"ALTER TABLE {self.rename_table} ALTER COLUMN LastName COMMENT 'new comment'"
        try:
            sql(sql_cmd)
            col_exist=sql(f"DESC {self.rename_table}").where("comment='new comment' AND col_name='LastName'").count()
            self.assertEqual(col_exist,1)
        except Exception as ex:
            msg={'command':'ALTER TABLE ALTER COLUMN COMMENT ','status':'fail'}
            self.fail(f"{msg}")
    
    def test_ddl_altertable_007_drop_table(self):
        """drop table"""
        table_name = f"{PREFIX}_drop_table_test_{SUFFIX}"
        sql_cmd = f"CREATE TABLE {table_name} (a STRING);"
        try:
            sql(sql_cmd)
            table_exist=sql(f"SHOW TABLES").where(f"tableName='{table_name}'").count()
            self.assertEqual(table_exist,1)
            sql(f"DROP TABLE {table_name}")
            table_exist=sql(f"SHOW TABLES").where(f"tableName='{table_name}'").count()
            self.assertEqual(table_exist,0)
        except Exception as ex:
            msg={'command':'DROP TABLE','status':'fail'}
            self.list_of_tables_to_remove.append(table_name)
            self.fail(f"{msg}")

    def test_ddl_altertable_008_remove_column(self):
        """Remove column supported with v2 tables."""
        sql_cmd = f"ALTER TABLE {self.rename_table} DROP COLUMN (LastName)"
        with self.assertRaises(AnalysisException,msg="ALTER TABLE DROP COLUMN - not supported with V1 table"):
            sql(sql_cmd)
       
    def test_ddl_altertable_009_rename_column(self):
        """Rename column - Supported only for V2 tables"""
        sql_cmd = f"ALTER TABLE {self.rename_table} RENAME COLUMN LastName To LName"
        with self.assertRaises(AnalysisException,msg="ALTER TABLE RENAME COLUMN - not supported with V1 table"):
            sql(sql_cmd)

   
    def test_ddl_altertable_010_replace_column(self):
        """Replace columns supported with v2 tables. """
        sql_cmd = f"ALTER TABLE {self.rename_table} REPLACE COLUMNS (name string, ID int COMMENT 'new comment')"
        with self.assertRaises(AnalysisException,msg="ALTER TABLE REPLACE COLUMNS"):
            sql(sql_cmd)

    @classmethod
    def tearDownClass(cls):
        """tear down"""
        for table in cls.list_of_tables_to_remove:
            try:
                sql(f"DROP TABLE IF EXISTS {table}")
            except Exception as e:
                print(f"table drop failed: {table}")



### DDL - Views

In [ ]:
import unittest
from pyspark.sql.utils import AnalysisException

class DDLViewTest(unittest.TestCase):
    """View test cases"""
    
    table_name = f"{PREFIX}_student_table_for_view_{SUFFIX}"
    view_name = f"{PREFIX}_student_table_view_{SUFFIX}"
   
    @classmethod
    def setUpClass(cls):
        sql_cmd = f"CREATE TABLE {cls.table_name} (id INT, fname STRING, lname STRING) \
                        PARTITIONED BY (age INT)"
        try:
            sql(sql_cmd)
            sql(f"INSERT INTO {cls.table_name} VALUES \
                 (1,'fa','la',10),(2,'fb','lb',20),(3,'fc','lc',30);")
        except Exception as ex:
            msg={'command':'View:Setup failed','status':'fail'}
            cls.fail(f"{msg}")
    
    def test_ddl_view_001_create_view(self):
        """create view"""
        sql_cmd = f"CREATE OR REPLACE VIEW  {self.view_name} (fname COMMENT 'First Name', lName COMMENT 'Last Name') \
                    COMMENT 'View for student table' \
                    AS SELECT fname, lname FROM {self.table_name};"
        try:
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE OR REPLACE VIEW','status':'fail'}
            self.fail(f"{msg}")
    
    def test_ddl_view_002_show_view(self):
        """show view"""
        try:
            view_exist=sql(f"SHOW VIEWS").filter(f"viewName='{self.view_name}'").count()
            self.assertEqual(view_exist,1)
        except Exception as ex:
            msg={'command':'SHOW VIEWS','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_view_003_show_view_like(self):
        """show view"""
        try:
            view_exist=sql(f"SHOW VIEWS LIKE '{self.view_name}*'").count()
            self.assertEqual(view_exist,1)
        except Exception as ex:
            msg={'command':'SHOW VIEWS LIKE','status':'fail'}
            self.fail(f"{msg}")

    def test_ddl_view_004_drop_view(self):
        """drop view"""
        try:
            sql(f"DROP VIEW {self.view_name}")
            view_exist=sql(f"SHOW VIEWS").filter(f"viewName='{self.view_name}'").count()
            self.assertEqual(view_exist,0)
        except Exception as ex:
            msg={'command':'DROP VIEW','status':'fail'}
            self.fail(f"{msg}")    
    
    
    @classmethod
    def tearDownClass(cls):
        """tear down"""
        try:
            sql(f"DROP TABLE IF EXISTS {cls.table_name}")
        except Exception as e:
            print(f"table drop failed: {cls.table_name}")


### DDL - Misc Table Command 
1. Truncate Table
2. Repair Table

In [ ]:
import unittest
from pyspark.sql.utils import AnalysisException

class DDLMiscTableOperations(unittest.TestCase):
    """miscellaneous table operations"""
    
    table_name = f"{PREFIX}_student_table_for_misc_operations_{SUFFIX}"
   
    @classmethod
    def setUpClass(cls):
        sql_cmd = f"CREATE TABLE {cls.table_name} (id INT, fname STRING, lname STRING) \
                        PARTITIONED BY (age INT)"
        try:
            sql(sql_cmd)
            sql(f"INSERT INTO {cls.table_name} VALUES \
                 (1,'fa','la',10),(11,'f1a','l1a',10),(2,'fb','lb',20),(3,'fc','lc',30);")
        except Exception as ex:
            msg={'command':'MiscTableOps:Setup failed','status':'fail'}
            cls.fail(f"{msg}")
    
    def test_ddl_misc_table_001_truncate_table_partition(self):
        """truncate table"""
        sql_cmd = f"TRUNCATE TABLE {self.table_name} PARTITION (age=20);"
        try:
            sql(sql_cmd)
            num_of_rows=sql(f"SELECT * FROM {self.table_name}").where("age=20").count()
            self.assertEqual(num_of_rows,0)
        except Exception as ex:
            msg={'command':'TRUNCATE TABLE PARTITION','status':'fail'}
            self.fail(f"{msg}")
    
    def test_ddl_misc_table_002_repair_table(self):
        """repair table recovers all the partitions in the directory of a table"""
        try:
            sql(f"MSCK REPAIR TABLE {self.table_name} DROP PARTITIONS")
        except Exception as ex:
            msg={'command':'MSCK REPAIR TABLE DROP PARTITIONS','status':'fail'}
            self.fail(f"{msg}")
    
    
    @classmethod
    def tearDownClass(cls):
        """tear down"""
        try:
            sql(f"DROP TABLE IF EXISTS {cls.table_name}")
        except Exception as e:
            print(f"table drop failed: {cls.table_name}")

### Execute Test Case

In [ ]:
import io
import xmlrunner
loader = unittest.TestLoader()
suite  = unittest.TestSuite()

# add tests to the test suite
suite.addTests(loader.loadTestsFromTestCase(DDLDatabaseTest))
suite.addTests(loader.loadTestsFromTestCase(DDLCreateTableTest))
suite.addTests(loader.loadTestsFromTestCase(DDLViewTest))
suite.addTests(loader.loadTestsFromTestCase(DDLAlterTableTest))
suite.addTest(loader.loadTestsFromTestCase(DDLMiscTableOperations))


# initialize a runner, pass it your suite and run it
out = io.BytesIO()
runner = xmlrunner.XMLTestRunner(output=out)
result = runner.run(suite)

## Report for Test

In [162]:
from pyspark.sql.functions import col, explode,isnull,from_json, expr, to_json, coalesce, lit
from pyspark.sql.types import StructType,StructField,StringType
import json
import xmltodict

dict_result=xmltodict.parse(out.getvalue())
json_result = json.loads(json.dumps(dict_result,indent=4).replace('@',''))
test_suites=json_result['testsuites']['testsuite']

df = spark.read.json(sc.parallelize([test_suites]))
fail_schema = StructType([
  StructField("command", StringType(), True),
  StructField("status", StringType(),  True)
])

test_cases_df= df.withColumn('ts',explode('testcase')).drop(col('testcase'))
if "failure:" in test_cases_df.schema.simpleString():
 explode_df= test_cases_df.withColumn('fail',from_json(col('ts.failure.message'),fail_schema)).drop(col('ts.failure'))
else:
 explode_df= test_cases_df.withColumn("fail",from_json(expr("to_json(named_struct('command', '', 'status', 'pass'))"),fail_schema))
 
df_test_result=explode_df.select(col("errors").alias("errorInSuite"),col("failures").alias("failedInSuite"),col("name").alias("suitename"),\
      "skipped",col("tests").alias("totalTest"), col("timestamp").alias("executionTime"),col("ts.name").alias("testCaseName"), \
       col("ts.time").alias("testCaseTime"),coalesce(col("fail.command"), lit("")).alias("failcommand"),coalesce(col("fail.status"), lit("pass")).alias("status"))

if (len(storage_account)>0 and len(result_container)>0):
    # save result to storage
    storage_path = f"abfs://{result_container}@{storage_account}.dfs.core.windows.net/{TEST_RUN_ID}/{PLATFORM}/{TEST_SUITE}"
    # write raw results
    df.write.parquet(f"{storage_path}/{RAW_RESULT_FILE_NAME}")
    # write transformed results
    df_test_result.write.parquet(f"{storage_path}/{RESULT_FILE_NAME}") 
else:
    print("configure storage path to store results")
    df_test_result.show(200,False)    